###### The python codes for Project1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

df = pd.read_csv('./listings.csv')
df.head()

#clean up some data, remove columns will all missing data or data input without id number
nd1 = df.dropna(how='all', axis=1)
nd1 = nd1.dropna(subset =['id'])


In [ ]:
#First got the count of listings for different neighbourhood

num_list = nd1.groupby(['host_neighbourhood'])['id'].count()
main_list= num_list[num_list>50]
main_list = main_list.sort_values(ascending = False)
main_list.plot(kind='bar', stacked=True, figsize=[10,6], colormap='winter',title='Boston Neighbourhoods Providing Most Airbnb Accomodations') 

Next we try to get the information of price and review score for different neibourhood. To make
the comparison more meaningful, we split the listing to 1 bed, 2 beds, 3 beds and 4+ beds

In [ ]:
#Since the 'price' of the dataset is in the format of currency, convert it to numeric format

import locale

locale.setlocale(locale.LC_ALL,'')
#print(df['price'])

aa = nd1['price']
bb = aa

#print(aa[aa.isnull()].count())

for i in range(len(aa)):
    bb[i] = locale.atof(aa[i].strip("$"))

nd1['price'] = pd.to_numeric(bb)
#print(bb)
print(nd1['price'])

In [ ]:
#Now group price and review scores by bed numbers and neighbourhoods

#aggregate bed >3 together
nd1['beds'].loc[nd1['beds']>3]=8.0


dprice=nd1.groupby(['beds','host_neighbourhood'])['price','review_scores_rating'].describe().reset_index()

dprice.set_index('host_neighbourhood')

print(dprice.columns)



In [ ]:
#Extract information for 1 bed, 2 beds, 3 beds and 4+ beds

#Got one bed with count >30
cond1 = dprice['beds']>0
cond2 = dprice['beds'] <1.5
cond3 = dprice['price']['count'] >30
cond = cond1 & cond2 &cond3
#print(cond)
nprice1 = dprice[cond]
#nprice1 = price[price['beds']>0 and price['beds'] <1.5]
#print(nprice1)

#Got two bed with count >30
cond1 = dprice['beds'] >1.5
cond2 = dprice['beds'] <2.5
cond3 = dprice['price']['count'] >20
cond = cond1 & cond2 &cond3
#print(cond)
nprice2 = dprice[cond]
#nprice1 = price[price['beds']>0 and price['beds'] <1.5]
#print(nprice2)

#Got three bed with count >10
cond1 = dprice['beds'] >2.5
cond2 = dprice['beds'] <3.5
cond3 = dprice['price']['count'] >10
cond = cond1 & cond2 &cond3
#print(cond)
nprice3 = dprice[cond]
#nprice1 = price[price['beds']>0 and price['beds'] <1.5]
#print(nprice3)

#Got 4+ bed with count >5
cond1 = dprice['beds'] >4
#cond2 = dprice['beds'] <3.5
cond3 = dprice['price']['count'] >5
cond = cond1 & cond3
#print(cond)
nprice4 = dprice[cond]
#nprice1 = price[price['beds']>0 and price['beds'] <1.5]
#print(nprice4)


In [ ]:
#Now plot the histogram for each catagory

plt.figure(figsize=[12,6])
plt.subplot(1,2,1)
pos = np.arange(nprice1.shape[0])
plt.bar(pos-0.2, nprice1['review_scores_rating']['mean'], align='center',color='g',width=0.35)
plt.bar(pos+0.2, nprice1['price']['mean'], align='center',color='r',width=0.35)
#ptick = pos
#for i in range(len(nprice1['host_neighbourhood'])):
#    ptick[i] = pos[i] -0.5*len(list(nprice1['host_neighbourhood'])[i])

plt.xticks(pos, nprice1['host_neighbourhood'],rotation=80)
plt.ylabel('Price')
plt.title('The price and review rating of 1 bed airbnb \nin Boston', alpha=0.8)

plt.subplot(1,2,2)
pos = np.arange(nprice2.shape[0])
plt.bar(pos-0.2, nprice2['review_scores_rating']['mean'], align='center',color='g',width=0.35)
plt.bar(pos+0.2, nprice2['price']['mean'], align='center',color='r',width=0.35)
plt.xticks(pos, nprice2['host_neighbourhood'],rotation=80)
plt.ylabel('Price')
plt.title('The price and rating of 2 beds airbnb \nin Boston', alpha=0.8)

plt.figure(figsize=[12,6])
plt.subplot(1,2,1)
pos = np.arange(nprice3.shape[0])
plt.bar(pos-0.2, nprice3['review_scores_rating']['mean'], align='center',color='g',width=0.35)
plt.bar(pos+0.2, nprice3['price']['mean'], align='center',color='r',width=0.35)
plt.xticks(pos, nprice3['host_neighbourhood'],rotation=80)
plt.ylabel('Price')
plt.title('The price and rating of 3 beds airbnb \nin Boston', alpha=0.8)

plt.subplot(1,2,2)
pos = np.arange(nprice4.shape[0])
plt.bar(pos-0.2, nprice4['review_scores_rating']['mean'], align='center',color='g',width=0.35)
plt.bar(pos+0.2, nprice4['price']['mean'], align='center',color='r',width=0.35)
plt.xticks(pos, nprice4['host_neighbourhood'],rotation=80)
plt.ylabel('Price')
plt.title('The price and rating of 4 or more bed airbnb \nin Boston', alpha=0.8)


In [ ]:
#start to prepare data for the modeling

def create_dummy_df(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables you want to dummy
    cat_cols - list of strings that are associated with names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics: 
            1. contains all columns that were not specified as categorical
            2. removes all the original columns in cat_cols
            3. dummy columns for each of the categorical columns in cat_cols
            4. if dummy_na is True - it also contains dummy columns for the NaN values
            5. Use a prefix of the column name with an underscore (_) for separating 
        '''

    df = pd.get_dummies(df, columns = cat_cols, prefix = cat_cols, prefix_sep = '_', drop_first= True, dummy_na = dummy_na)

    return df



def prep_data(df,resp, dummy_na):
    '''
    INPUT
    df - pandas dataframe 
    resp - the response column name
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    including dummy columns for all the categorical variables in X with the original columns dropped
    y - the corresponding response vector
    
    
    '''
    #Drop all the rows with no response value
    df = df.dropna(subset = [resp])
    
    #Drop those columns with all missing data
    df = df.dropna(how ='all',axis=1)
    
    #seperate response vector and the prediction matrix
    y = df[resp]
    X = df.drop(columns = resp)
    
    #For each numeric variable in X, fill the column with the mean value of the column.
    for col in X.columns:
        if X[col].dtypes in ['int64',  'float64'] :
            X[col] = X[col].fillna(X[col].mean())
    
    #Create dummy columns for all the categorical variables in X, drop the original columns
    X_cat = X.select_dtypes(include = ['object']).copy()
    X_cat_cols = X_cat.columns
    
    X = create_dummy_df(df, X_cat_cols, dummy_na)
    #X = pd.get_dummies(X, columns = X_cat_cols, prefix = X_cat_cols, prefix_sep = '_', drop_first= True, dummy_na = False)
    
    #fill the NAN in X with mean of the columns
    fill_by_mean = lambda col:col.fillna(col.mean())
    X = X.apply(fill_by_mean, axis=0)
    
    return X, y
    
#Use the function to create X and y, if I drop those urls from the data, the test score is much better
nd1 = nd1.drop(columns = ['listing_url','thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url'])
X, y = prep_data(nd1,'review_scores_rating', dummy_na=False) 
#X, y = prep_data(nd1,'price', dummy_na=False)

In [ ]:
#Create a linear model for the data and test the model

def proj1_linear_mod(X, y, test_size=.3, rand_state=42):
    '''
    INPUT:
    X - a dataframe holding all the variables of interest as predictor
    y - a vector holding response variable 
    
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''
    X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = test_size, random_state = rand_state)
    
    lm_model = LinearRegression(normalize = True)
    lm_model.fit(X_train,y_train)
    
    y_test_pred = lm_model.predict(X_test)
    #print(y_test_pred)
    
    y_train_pred = lm_model.predict(X_train)
    
    test_score = r2_score(y_test, y_test_pred)
    train_score = r2_score(y_train, y_train_pred)

    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

#test the result
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = proj1_linear_mod(X, y, test_size=.3, rand_state=42)

In [ ]:
#Check the impact of the coefficients

def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df

#Use the function
coef_df = coef_weights(lm_model.coef_, X_train)

#A quick look at the top results
coef_df.head(20)